<a href="https://colab.research.google.com/github/jvataidee/pdi_python/blob/main/conversao_radiancia_de_superficie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Conversão de Radiância para Reflectância de Superfície**

## **Criando ambiente para PDI com Python**

### Instalando Miniconda
[MINICONDA](https://docs.conda.io/en/latest/miniconda.html)

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local

### Definir ambiente

In [2]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

### Testando comando conda
[CONDA](https://docs.conda.io/en/latest/)

In [ ]:
!conda

### Instalando biblioteca rsgslib
[RSGISLIB](https://github.com/remotesensinginfo/rsgislib)

In [ ]:
!conda install -c conda-forge rsgislib -y

### Instalando biblioteca Py6s
[PY6s](https://py6s.readthedocs.io/en/latest/installation.html)

In [ ]:
!conda install -c conda-forge Py6S -y

### Instalando biblioteca spectral
[SPECTRAL](https://github.com/stoplightio/spectral)

In [ ]:
!pip install spectral

## **Inciando o processamento**

In [ ]:
from spectral import imshow
import matplotlib.pyplot as plt
import numpy as np
import tifffile as tif
from Py6S import *
import math

### Tefinindo parâmetros

In [ ]:
# Definindo o ângulo zenital solar
solar_z = 90 - 40.82290583

# Definindo a altitude
alt = 1.1

# Criando uma instância do objeto SixS
s = SixS()

# Configurando os componentes atmosféricos
s.atmos_profile = AtmosProfile.FromLatitudeAndDate(15, '2001-07-20')  # Perfil atmosférico baseado na latitude e data
s.aero_profile = AeroProfile.Continental  # Perfil de aerossóis utilizado (perfil "Continental")

# Configurando a geometria Terra-Sol-satélite
s.geometry = Geometry.User()
s.geometry.view_z = 0  # Ângulo de visada (sempre NADIR)
s.geometry.solar_z = solar_z  # Ângulo zenital solar
s.geometry.month = 7  # Mês utilizado para o cálculo da distância Terra-Sol
s.geometry.day = 20  # Dia utilizado para o cálculo da distância Terra-Sol

# Configurando as altitudes
s.altitudes.set_sensor_satellite_level()  # Altitude do sensor do satélite
s.altitudes.set_target_custom_altitude(alt)  # Altitude do alvo personalizada

### Criando função de ganho

In [ ]:
def fun_ganho(bandname):
    # Dicionário de seleção das bandas e seus comprimentos de onda correspondentes
    bandSelect = {
        'B1': PredefinedWavelengths.LANDSAT_ETM_B1,
        'B2': PredefinedWavelengths.LANDSAT_ETM_B2,
        'B3': PredefinedWavelengths.LANDSAT_ETM_B3,
        'B4': PredefinedWavelengths.LANDSAT_ETM_B4,
        'B5': PredefinedWavelengths.LANDSAT_ETM_B5,
        'B7': PredefinedWavelengths.LANDSAT_ETM_B7,
    }

    # Retornando o comprimento de onda correspondente à banda especificada
    return Wavelength(bandSelect[bandname])

In [ ]:
s.outputs.tran

In [ ]:
def radiancia(bandname, img):
    # Coeficientes ESUN para Landsat 7
    ESUN_L7 = [1970, 1842, 1547, 1044, 225.7, 82.06]

    # Mapeamento das bandas para os coeficientes ESUN correspondentes
    ESUN_BAND = {
        'B1': ESUN_L7[0],
        'B2': ESUN_L7[1],
        'B3': ESUN_L7[2],
        'B4': ESUN_L7[3],
        'B5': ESUN_L7[4],
        'B7': ESUN_L7[5],
    }

    # Cálculo do fator de correção angular solar
    solar_angle_correction = math.cos(solar_z) ** 2

    # Cálculo do multiplicador para converter radiância para irradiância
    multiplier = ESUN_BAND[bandname] * solar_angle_correction / (math.pi * 1.0161264 ** 2)

    # Radiância no sensor
    rad = img * multiplier

    # Configurando o comprimento de onda do SixS para a banda atual
    s.wavelength = fun_ganho(bandname)

    # Executando o modelo SixS
    s.run()

    # Extraindo as saídas do SixS
    Edir = s.outputs.direct_solar_irradiance  # Irradiância solar direta
    Edif = s.outputs.diffuse_solar_irradiance  # Irradiância solar difusa
    Lp = s.outputs.atmospheric_intrinsic_radiance  # Radiância de caminho
    absorb = s.outputs.trans['global_gas'].upward  # Transmitância de absorção
    scatter = s.outputs.trans['total_scattering'].upward  # Transmitância de espalhamento
    tau2 = absorb * scatter  # Transmitância total

    # Conversão da radiância para reflectância de superfície
    ref = ((rad - Lp) * math.pi) / (tau2 * (Edir + Edif))

    return ref

In [ ]:
# Transmitância de espalhamento total
s.outputs.trans['total_scattering'].upward

In [ ]:
# Transmitância de gás global
s.outputs.trans['global_gas'].upward

In [ ]:
# Irradiância solar direta
s.outputs.direct_solar_irradiance

In [ ]:
# Irradiância solar difusa
s.outputs.diffuse_solar_irradiance

### Importando Imagens

In [ ]:
#from google.colab import drive
#drive.mount()

In [ ]:
L7 = tif.imread('/content/drive/MyDrive/pdi_python/21 - PDI com Python/01 - Pré-Processamento/L71221071_07120010720_DN.tif')

In [ ]:
# Criando a lista de bandas
lista_bandas = ['B1','B2','B3','B4','B5','B7']

In [ ]:
# Criando um array vazio com a mesma forma (shape) da imagem L7
f = np.zeros_like(L7)

# Iterando sobre as bandas da imagem L7
for i in range(L7.shape[2]):
    # Chamando a função radiancia para calcular a reflectância de superfície para a banda atual
    f[:, :, i] = radiancia(l[i], L7[:, :, i])


In [ ]:
imshow(f,(2,3,1),figsize=(12,8))

In [ ]:
imshow(f[:,:,4],figsize=(12,8))

In [ ]:
# Criação do histograma para a banda de reflectância de superfície (f)
plt.hist(f[:, :, 0].flatten(), bins=50)

# Criação do histograma para a banda original da imagem Landsat (L7)
plt.hist(L7[:, :, 0].flatten(), bins=50)

# Exibição dos histogramas
plt.show()